In [1]:
!python3 --version

Python 3.10.12


#### fix for could not import -lmza
https://github.com/lucidrains/imagen-pytorch/issues/92 

#### Fix for Can't import datasets AttributeError: partially initialized module 'datasets' has no attribute 'utils' (most likely due to a circular import)

Restart juypyter and kernel


In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
import helper
import time
helper.check_gpu_capacity()

GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [4]:
from datasets import load_dataset, load_from_disk
from random import randrange



In [5]:

# Load dataset from the hub
dataset = load_from_disk("generated-dataset/peft-vicuna-int4-instruct-generation")

dataset


Dataset({
    features: ['instruction', 'response'],
    num_rows: 709
})

In [6]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
How to develop a recipe for a base powder?
--> response
Powder tab shows levels on nutrients after wet phase processing losses Usually vitamins and other ingredients which are heat sensitive will have fixed losses after basepowder production. 

Warranty tab shows levels after process & shelf life losses. This can be depicted as product (g/100g product) or on ref. nutr (g/100ml product). This tab must be used to optimize nutrients on ref. nutr during recipe development as mentioned in project brief/framework. 

Diluted tab gives rounded figures when “dilution factor” is filled in header folder


Contribution list. When clicking on ingredient this field shows composition of the specific ingredient. When clicking on nutrient this field gives the contribution overview of all ingredients linked to the selected nutrient. See below figure on explanation of columns

Figure 6.2: explanation columns Contribution list



6.2	Recipe development



Built the recipe in the optimizati

In [7]:
# list(filter(lambda x :x != '', list(map(lambda x:((x['category'], len(x['context'])) if len(x['context']) > 5000 else ''), dataset))))

Based on our use case, we want to focus on 'general_qa' and  'information_extraction tasks.,

In [8]:
dataset = dataset.filter(lambda x: x['instruction'] != '')

# print(set(dataset['instruction']))
dataset

Loading cached processed dataset at /home/ubuntu/ai-project/generated-dataset/peft-vicuna-int4-instruct-generation/cache-98139ac2c7920251.arrow


Dataset({
    features: ['instruction', 'response'],
    num_rows: 709
})

In [9]:
sample_ds = dataset[randrange(len(dataset))]

for key, value in sample_ds.items():
    print(f"--> {key}")
    print("====================")
    print(value)
    print("====================")

--> instruction
What is the purpose of this instruction?
--> response
IN.15.02 Instruction PSP and PSL creation in Artemis 

Purpose

The objective of this instruction is to explain how to create a Product Specification for Liquid and Powder Products.  

Scope

This instruction applies for every PD Milks Recipe that has been sent from Bestmix to Artemis. Clinical Studies Recipes and Third Parties recipes are out of this scope.

References

All references in their currently valid version.

PR.15 Procedure Official PD Documents.

ARTEMIS KIT ELN MILK and MEDICAL PRODUCTS

Instruction

Substance database: PSP/PSL	


Click on the transaction: 

Type the Output code and then ENTER

Click on Output Variant:  	

One-click on Baby&Medical Specific Reports and select the report (1) and then Click on Output button (2)

Choose appropriate template and Risk Group: 



Choose legislation:

Choose “Allow” if you get a warning message:

Save document as XLS file: 

Manual rework of the file:



Only 

Split the dataset into train, eval, and test sets

In [10]:

shuffled_dataset = dataset.shuffle(seed=42)
shuffled_dataset = shuffled_dataset.train_test_split(test_size=0.1)

train_dataset = shuffled_dataset['train']
test_dataset = shuffled_dataset['test']

train_eval_dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = train_eval_dataset['train']
eval_dataset = train_eval_dataset['test']


# Print the sizes of the train, eval, andtest sets
print("Dataset size:", len(dataset))
print("Train size:", len(train_dataset))
print("Eval size:", len(eval_dataset))
print("Test size:", len(test_dataset))

Loading cached shuffled indices for dataset at /home/ubuntu/ai-project/generated-dataset/peft-vicuna-int4-instruct-generation/cache-efe3177fc26de7b0.arrow


Dataset size: 709
Train size: 510
Eval size: 128
Test size: 71


### Preprocess the Dialog-Summary Dataset
We need to convert the instruction-response (prompt-response) pairs into explicit instructions for the LLM. 

Prepend an instruction to the start of the dialog with Answer this question and to the start of the question with question as follows:

Training prompt (Question):
Answer this question.

    What is the capital of France?

Answer: 
Training response (answer):

    The capital of France is Paris.




In [11]:
def format_instruction(sample):
	return f"""### Instruction:
            Answer this question.
            
            ### Question:
            {sample['instruction']}

            ### Answer:
            {sample['response']}
        """


Let's test our formatting function on a random example.

In [12]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))

## Do I need to add the context in the format_instruction?????

### Instruction:
            Answer this question.
            
            ### Question:
            Based on the following text, please provide a summary of the sensory test results for the accelerated and ambient shelf life studies.

            ### Answer:
            Author
Start date SL test
End date SL test
Date final report
Project/Product
Stage
PDS code
Packaging / format
Shelf life test code
Storage conditions    Ambient 25 °C
                      Accelerated:  45°C  /75%RH
                      Open Can 25°C

Packing supply point
Batch codes

General Project Information

Product Information

Provide general information for the product such as product name, stage and if any specific parameter.ingredient  in the recipe might impact the shelf life (e.g. high LCPs)

Reason for shelf life testing 

Reason for this shelf life  study is for EU China PD team to follow up with the Product name sensorial performance. In parallel with this SL study, China team in Shanghai is conductin

In [13]:
print(format_instruction(sample_ds))

### Instruction:
            Answer this question.
            
            ### Question:
            What is the purpose of this instruction?

            ### Answer:
            IN.15.02 Instruction PSP and PSL creation in Artemis 

Purpose

The objective of this instruction is to explain how to create a Product Specification for Liquid and Powder Products.  

Scope

This instruction applies for every PD Milks Recipe that has been sent from Bestmix to Artemis. Clinical Studies Recipes and Third Parties recipes are out of this scope.

References

All references in their currently valid version.

PR.15 Procedure Official PD Documents.

ARTEMIS KIT ELN MILK and MEDICAL PRODUCTS

Instruction

Substance database: PSP/PSL	


Click on the transaction: 

Type the Output code and then ENTER

Click on Output Variant:  	

One-click on Baby&Medical Specific Reports and select the report (1) and then Click on Output button (2)

Choose appropriate template and Risk Group: 



Choose legislation:



### Instruction-tune Llama 2 using trl and the SFTTrainer

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False
# COMMENT IN TO USE FLASH ATTENTION
# replace attention with flash attention
# if torch.cuda.get_device_capability()[0] >= 8:
#     from utils.llama_patch import replace_attn_with_flash_attn
#     print("Using flash attention")
#     replace_attn_with_flash_attn()
#     use_flash_attention = True

helper.check_gpu_capacity()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 22511 MB
  Used Memory: 4 MB


In [15]:
# Hugging Face model id
model_llama_2_7b_id = "lmsys/vicuna-7b-v1.5" # non-gated
# model_id = "meta-llama/Llama-2-7b-hf" # gated

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [16]:
# Load model and tokenizer
model_llama_2_7b = AutoModelForCausalLM.from_pretrained(model_llama_2_7b_id, 
                                                        quantization_config=bnb_config, 
                                                        use_cache=False, 
                                                        device_map="auto")
tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)
model_llama_2_7b.hf_device_map

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 1,
 'model.layers.7': 1,
 'model.layers.8': 1,
 'model.layers.9': 1,
 'model.layers.10': 1,
 'model.layers.11': 1,
 'model.layers.12': 1,
 'model.layers.13': 1,
 'model.layers.14': 1,
 'model.layers.15': 1,
 'model.layers.16': 2,
 'model.layers.17': 2,
 'model.layers.18': 2,
 'model.layers.19': 2,
 'model.layers.20': 2,
 'model.layers.21': 2,
 'model.layers.22': 2,
 'model.layers.23': 2,
 'model.layers.24': 2,
 'model.layers.25': 2,
 'model.layers.26': 3,
 'model.layers.27': 3,
 'model.layers.28': 3,
 'model.layers.29': 3,
 'model.layers.30': 3,
 'model.layers.31': 3,
 'model.norm': 3,
 'lm_head': 3}

In [17]:
tokenizer_llama_2_7b.model_max_length

4096

In [18]:
def format_instruction(sample):
	return f"""### Instruction:
            Answer this question.
            
            ### Question:
            {sample['instruction']}

            ### Answer:
            {sample['response']}
        """


In [19]:
def tokenize_function(sample, tokenizer):
    prompt = f"""### Instruction:
            Answer this question.
            
            ### Question:
            {sample['instruction']}

        """
    
    sample['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample["response"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return sample

tokenized_train_dataset = train_dataset.map(
    tokenize_function, 
    fn_kwargs={"tokenizer": tokenizer_llama_2_7b},
    # batched=True
)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(['instruction', 'response'])

tokenized_eval_dataset = eval_dataset.map(
    tokenize_function, 
    fn_kwargs={"tokenizer": tokenizer_llama_2_7b},
    # batched=True
)
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(['instruction', 'response'])

tokenized_test_dataset = test_dataset.map(
    tokenize_function, 
    fn_kwargs={"tokenizer": tokenizer_llama_2_7b},
    # batched=True
)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(['instruction', 'response'])

print('The output dataset is ready for fine-tuning.')

Map:   0%|          | 0/510 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

The output dataset is ready for fine-tuning.


In [20]:

model_llama_2_7b.config.pretraining_tp = 1

# Validate that the model is using flash attention, by comparing doc strings
if use_flash_attention:
    from utils.llama_patch import forward
    assert model_llama_2_7b.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, "Model is not using flash attention"


# tokenizer_llama_2_7b = AutoTokenizer.from_pretrained(model_llama_2_7b_id)
tokenizer_llama_2_7b.pad_token = tokenizer_llama_2_7b.eos_token
tokenizer_llama_2_7b.padding_side = "right"

The SFTTrainer supports a native integration with peft, which makes it super easy to efficiently instruction tune LLMs. We only need to create our LoRAConfig and provide it to the trainer.

Setup the PEFT/LoRA model for Fine-Tuning

In [21]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"



In [22]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM", # Llama 2
)

# prepare model for training
model_llama_2_7b_peft = prepare_model_for_kbit_training(model_llama_2_7b)
model_llama_2_7b_peft = get_peft_model(model_llama_2_7b_peft, peft_config)
print(print_number_of_trainable_model_parameters(model_llama_2_7b_peft))


trainable model parameters: 33554432
all model parameters: 3533967360
percentage of trainable model parameters: 0.95%


### Train PEFT Adapter
Define training arguments and create Trainer instance.

Before we can start our training we need to define the hyperparameters (TrainingArguments) we want to use.

In [23]:
from transformers import TrainingArguments

peft_training_args = TrainingArguments(
    output_dir="models/peft-vicuna-int4-fine-tuned",
    num_train_epochs=5,
    per_device_train_batch_size=6 if use_flash_attention else 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False # disable tqdm since with packing values are in correct
)


In [24]:
from trl import SFTTrainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model_llama_2_7b_peft,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer_llama_2_7b,
    packing=True,
    formatting_func=format_instruction,
    args=peft_training_args,
)
peft_training_args.device

device(type='cuda', index=0)

In [25]:
helper.check_gpu_capacity()

helper.free_gpu_memory()
helper.memory_stats()


GPU Information:
GPU 0:
  Total Memory: 23028 MB
  Free Memory: 20771 MB
  Used Memory: 1744 MB
GPU 1:
  Total Memory: 23028 MB
  Free Memory: 21095 MB
  Used Memory: 1420 MB
GPU 2:
  Total Memory: 23028 MB
  Free Memory: 21095 MB
  Used Memory: 1420 MB
GPU 3:
  Total Memory: 23028 MB
  Free Memory: 20771 MB
  Used Memory: 1744 MB
GPU Memory Used: 1.12 GB
GPU Memory Cached: 1.16 GB


In [26]:
%%time

# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()
# After training, access the path of the best checkpoint like this
best_ckpt_path = trainer.state.best_model_checkpoint

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.346100
20,2.228000
30,2.168200
40,2.067200
50,2.005600
60,1.963700
70,1.855500


CPU times: user 15min 36s, sys: 17min 9s, total: 32min 45s
Wall time: 32min 43s


## Test Model and run Inference

In [27]:
if use_flash_attention:
    # unpatch flash attention
    from utils.llama_patch import unplace_flash_attn_with_attn
    unplace_flash_attn_with_attn()

import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model_dir = "models/peft-vicuna-int4-fine-tuned"

# load base LLM model and tokenizer
model = AutoPeftModelForCausalLM.from_pretrained(
    peft_training_args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Let’s load the dataset again with a random sample to try to generate an instruction.

In [28]:
# from datasets import load_dataset
# from random import randrange


# # Load dataset from the hub and get a sample
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
# sample = dataset[randrange(len(dataset))]

# prompt = f"""### Instruction:
# Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

# ### Input:
# {sample_ds['response']}

# ### Response:
# """

prompt = f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
4.1.	UV-Light
Together with oxygen, UV-light can produce a singlet oxygen which can form directly a hydroperoxide from a unsaturated lipid (see Figure 3). Usually during the processing of powders, prevention of UV-light intrusion is being controlled. Furthermore, it depends on the package used, but as well how the consumer stores the product. 
4.2.	Oxygen
Reaction with oxygen can take at various sites in the lipid oxidation chain reaction (see Figure 3). Exclusion of oxygen means mainly flushing inert gases typically nitrogen (or argon) to displace oxygen, packaging in vacuum or in modified atmosphere packaging (MAP). Oxygen scavengers can be employed to completely remove residual oxygen from the system. 
4.3.	Heat
Higher temperatures speed up the lipid oxidation reaction rates. Although the product should be prevented from heat, during production heat is needed to: 
1.	be able to handle ingredients (e.g. melting of fat, best done under nitrogen)
2.	get a product microbiologically stable
3.	spray dry the product
Clearly heat is needed during production but where possible it should be minimized to prevent lipid oxidation. 

### Response:
"""

prompt = """ ### Instruction:
Answer the questions below: If you don't know, simply say I don't know; Don't make up answers.

### Question:
What are the main factors that affect the lipid oxidation of food?

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=450, 
                         do_sample=True, 
                         top_p=0.9,temperature=0.9
                        )

outputs = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]#[len(prompt):]

# print(f"Prompt:\n{sample_ds['response']}\n")
print(f"Generated instruction:\n{outputs}")
# print(f"Ground truth:\n{sample_ds['instruction']}")


Generated instruction:
 ### Instruction:
Answer the questions below: If you don't know, simply say I don't know; Don't make up answers.

### Question:
What are the main factors that affect the lipid oxidation of food?

### Response:

Food oxidation is a complex process that involves multiple factors and mechanisms. The main factors affecting the lipid oxidation of food can be classified into two main categories: internal factors and external factors (table 1).

Table 1. Main factors affecting the lipid oxidation of food.



Internal factors:

2.1 1. Lipid composition of the food matrix
2.1.1 1.1 Saturated fatty acids (SFA)
2.1.2 1.2 Unsaturated fatty acids (UFA)
2.1.3 1.3 Conjugated linoleic acid (CLA)
2.1.4 1.4 ω-3 fatty acids (EPA, DHA)
2.1.5 1.5 Trans fatty acids
2.1.6 1.6 Polyunsaturated fatty acids (PUFA)
2.1.7 1.7 Sterols
2.1.8 1.8 Sterolins
2.1.9 1.9 Cholesterol
2.1.10 1.10 Phytosterols

2.2 2. Microbial activity
2.2.1 2.1 Microbial lipases
2.2.2 2.2 Microbial lipoxidases
2.2.3 

### Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 50 instructions and responses to save time). 

In [31]:
test_dataset

Dataset({
    features: ['instruction', 'response'],
    num_rows: 69
})

In [33]:
test_dataset[41]['instruction']

'Who is responsible for considering the nutritional losses during shelf life?'

In [38]:
nn = 41

instruction = test_dataset[nn]['instruction']
print(instruction)
response = test_dataset[nn]['response']

prompt = f"""### Instruction:
Answer this question.

### Question:
{instruction}

### Answer:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=400, 
                         do_sample=True, top_p=0.9, temperature=0.2)

# print(f"Prompt:\n{sample_ds['response']}\n")
generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
# generated_instruction = generated_instruction.split('\n')[0].strip()
generated_instruction

Who is responsible for considering the nutritional losses during shelf life?


'The shelf life of the product is determined by the sensorial and nutritional parameters. The sensorial parameters are assessed by the sensorial panel and the nutritional parameters are assessed by the nutritional lab. The nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutritional lab is responsible for considering the nutritional losses during shelf life.\n\nThe nutrition

In [36]:
test_dataset[nn]['response']

'step 4. Shelf life Losses\n\n\n\nResponsible                                                                                                                                                                                                                                                                                                                                                                                                                  PD\nThe 4th building block to set-up the nutritional range of a recipe is considering the nutritional losses during shelf life;                                                                                                                                                                                                                                                                                                  The 4th building block to set-up the nutritional range of a recipe is considering the nutritional losses during shelf life;'

In [39]:
from pprint import pprint

pprint(generated_instruction)

('The shelf life of the product is determined by the sensorial and nutritional '
 'parameters. The sensorial parameters are assessed by the sensorial panel and '
 'the nutritional parameters are assessed by the nutritional lab. The '
 'nutritional lab is responsible for considering the nutritional losses during '
 'shelf life.\n'
 '\n'
 'The nutritional lab is responsible for considering the nutritional losses '
 'during shelf life.\n'
 '\n'
 'The nutritional lab is responsible for considering the nutritional losses '
 'during shelf life.\n'
 '\n'
 'The nutritional lab is responsible for considering the nutritional losses '
 'during shelf life.\n'
 '\n'
 'The nutritional lab is responsible for considering the nutritional losses '
 'during shelf life.\n'
 '\n'
 'The nutritional lab is responsible for considering the nutritional losses '
 'during shelf life.\n'
 '\n'
 'The nutritional lab is responsible for considering the nutritional losses '
 'during shelf life.\n'
 '\n'
 'The nutritio

In [41]:
import pandas as pd


instruction_test = dataset['instruction'][50:100]
response_test = dataset['response'][50:100]

generated_instruction_list = []
ground_truth_list = []

for instruction, response in zip(instruction_test, response_test):
    prompt = f"""### Instruction:
Answer this question.

### Input:
{instruction}

### Response:
"""

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    # with torch.inference_mode():
    outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9,temperature=0.2)
    
    # print(f"Prompt:\n{sample_ds['response']}\n")
    generated_instruction = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    generated_instruction = generated_instruction.split('\n')[0].strip()

    generated_instruction_list.append(generated_instruction)
    ground_truth_list.append(instruction)

zipped_summaries = list(zip(generated_instruction_list, ground_truth_list))
 
df = pd.DataFrame(zipped_summaries, columns = ['generated_instruction_list', 'ground_truth_list'])
df

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

In [37]:
for index, row in df[40:50].iterrows():
    print('GI: ', row['generated_instruction_list'])

    print('GT: ',row['ground_truth_list'], '\n')

GI:  What is the difference between Viola toeria and Samphor?
GT:  Identify which instrument is string or percussion: Samphor, Viola toeria 

GI:  What are the things found inside a house?
GT:  If we were playing a game where we had to identify things that can be found inside a house, which of these would we call out: car, chair, table, park, cloud, microwave. 

GI:  What are the different types of beer?
GT:  Classify the following as either dark-colored beers or light colored beers: porter, pilsner, stout, amber, lager 

GI:  
GT:  Give me a bulleted list of the 5 highest mountains in the world and their respective heights in meters 

GI:  • When was Daniel Leavitt born?
GT:  Extract all of the dates mentioned in this paragraph and list them using bullets in the format {Date} - {Description} 

GI:  What products does Apple sell?
GT:  Which products apple sell? 

GI:  What is the moral of the story?
GT:  Write a short story about a person who discovers a hidden room in their house. The

In [38]:
!pip install evaluate==0.4.0 rouge_score==0.1.2 --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Compute ROUGE score for this subset of the data. 

In [39]:
import evaluate

rouge = evaluate.load('rouge')

model_results = rouge.compute(
    predictions=generated_instruction_list,
    references=ground_truth_list[0:len(generated_instruction_list)],
    use_aggregator=True,
    use_stemmer=True,
)

print('MODEL:')
model_results_df = pd.DataFrame(model_results.items(), columns=['Metric', 'Value'])
model_results_df

MODEL:


,Metric,Value
0,rouge1,0.467613
1,rouge2,0.297052
2,rougeL,0.436774
3,rougeLsum,0.434922
